<a href="https://colab.research.google.com/github/jesus23lk/Python-Notebooks/blob/main/12_1_Self_Attention.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Notebook 12.1: Self Attention**

This notebook builds a self-attention mechanism from scratch, as discussed in section 12.2 of the book.

Work through the cells below, running each cell in turn. In various places you will see the words "TO DO". Follow the instructions at these places and make predictions about what is going to happen or write code to complete the functions.

Contact me at udlbookmail@gmail.com if you find any mistakes or have any suggestions.



In [ ]:
import numpy as np
import matplotlib.pyplot as plt

The self-attention mechanism maps $N$ inputs $\mathbf{x}_{n}\in\mathbb{R}^{D}$ and returns $N$ outputs $\mathbf{x}'_{n}\in \mathbb{R}^{D}$.  



In [ ]:
# Set seed so we get the same random numbers
np.random.seed(3)
# Number of inputs
N = 3
# Number of dimensions of each input
D = 4
# Create an empty list
all_x = []
# Create elements x_n and append to list
for n in range(N):
  all_x.append(np.random.normal(size=(D,1)))
# Print out the list
print(all_x)


We'll also need the weights and biases for the keys, queries, and values (equations 12.2 and 12.4)

In [ ]:
# Set seed so we get the same random numbers
np.random.seed(0)

# Choose random values for the parameters
omega_q = np.random.normal(size=(D,D))
omega_k = np.random.normal(size=(D,D))
omega_v = np.random.normal(size=(D,D))
beta_q = np.random.normal(size=(D,1))
beta_k = np.random.normal(size=(D,1))
beta_v = np.random.normal(size=(D,1))

Now let's compute the queries, keys, and values for each input

In [ ]:
# Make three lists to store queries, keys, and values
all_queries = []
all_keys = []
all_values = []

# For every input
for x in all_x:
    # TODO -- compute the keys, queries and values.
    query = np.dot(omega_q, x) + beta_q
    key = np.dot(omega_k, x) + beta_k
    value = np.dot(omega_v, x) + beta_v

    all_queries.append(query)
    all_keys.append(key)
    all_values.append(value)


We'll need a softmax function (equation 12.5) -- here, it will take a list of arbitrary numbers and return a list where the elements are non-negative and sum to one


In [ ]:
def softmax(items_in):
    # Compute the exponentials and normalize by the sum of exponentials
    exps = np.exp(items_in - np.max(items_in))  # for numerical stability
    items_out = exps / np.sum(exps)

    return items_out


Now compute the self attention values:

In [ ]:
# Create empty list for output
all_x_prime = []

# For each output
for n in range(N):
    # Create list for dot products of query N with all keys
    all_km_qn = []
    # Compute the dot products
    for key in all_keys:
        # Compute the dot product between the query and each key
        dot_product = np.dot(all_queries[n].T, key)

        # Store dot product
        all_km_qn.append(dot_product)

    # Compute attention using softmax
    attention = softmax(all_km_qn)
    print("Attentions for output ", n)
    print(attention)

    # Compute a weighted sum of all of the values according to the attention
    x_prime = np.sum([attention[i] * all_values[i] for i in range(len(all_values))], axis=0)

    all_x_prime.append(x_prime)


Now let's compute the same thing, but using matrix calculations.  We'll store the $N$ inputs $\mathbf{x}_{n}\in\mathbb{R}^{D}$ in the columns of a $D\times N$ matrix, using equations 12.6 and 12.7/8.

Note:  The book uses column vectors (for compatibility with the rest of the text), but in the wider literature it is more normal to store the inputs in the rows of a matrix;  in this case, the computation is the same, but all the matrices are transposed and the operations proceed in the reverse order.

In [ ]:
# Define softmax operation that works independently on each column
def softmax_cols(data_in):
  # Exponentiate all of the values
  exp_values = np.exp(data_in) ;
  # Sum over columns
  denom = np.sum(exp_values, axis = 0);
  # Replicate denominator to N rows
  denom = np.matmul(np.ones((data_in.shape[0],1)), denom[np.newaxis,:])
  # Compute softmax
  softmax = exp_values / denom
  # return the answer
  return softmax

In [ ]:
def self_attention(X, omega_v, omega_q, omega_k, beta_v, beta_q, beta_k):
    # Compute queries, keys, and values
    queries = np.dot(X, omega_q.T) + beta_q
    keys = np.dot(X, omega_k.T) + beta_k
    values = np.dot(X, omega_v.T) + beta_v

    # Compute dot products and apply softmax to calculate attentions
    dot_products = np.dot(queries, keys.T)
    attentions = softmax(dot_products)

    # Weight values by attentions
    X_prime = np.dot(attentions, values)

    return X_prime


In [ ]:
# Copy data into matrix
X = np.zeros((D, N))
X[:,0] = np.squeeze(all_x[0])
X[:,1] = np.squeeze(all_x[1])
X[:,2] = np.squeeze(all_x[2])

# Run the self attention mechanism
X_prime = self_attention(X,omega_v, omega_q, omega_k, beta_v, beta_q, beta_k)

# Print out the results
print(X_prime)

If you did this correctly, the values should be the same as above.

TODO:  

Print out the attention matrix
You will see that the values are quite extreme (one is very close to one and the others are very close to zero.  Now we'll fix this problem by using scaled dot-product attention.

In [ ]:
def scaled_dot_product_self_attention(X, omega_v, omega_q, omega_k, beta_v, beta_q, beta_k):
    # Compute queries, keys, and values
    queries = np.dot(X, omega_q.T) + beta_q
    keys = np.dot(X, omega_k.T) + beta_k
    values = np.dot(X, omega_v.T) + beta_v

    # Compute scaled dot products (divide by sqrt of key dimension size)
    d_k = keys.shape[1]  # dimension of the keys
    dot_products = np.dot(queries, keys.T) / np.sqrt(d_k)

    # Apply softmax to calculate attentions
    attentions = softmax(dot_products)

    # Weight values by attentions
    X_prime = np.dot(attentions, values)

    return X_prime


In [ ]:
# Run the self attention mechanism
X_prime = scaled_dot_product_self_attention(X,omega_v, omega_q, omega_k, beta_v, beta_q, beta_k)

# Print out the results
print(X_prime)

TODO -- Investigate whether the self-attention mechanism is covariant with respect to permutation.
If it is, when we permute the columns of the input matrix $\mathbf{X}$, the columns of the output matrix $\mathbf{X}'$ will also be permuted.
